In [ ]:
from Functions import*
import matplotlib.pyplot as plt

# connect to the LCR meter
ser = DeviceConnect('COM9')

# define your frequencies
freq = Frequencies(110, 1000, 3)

# set which measurements you'd like to use as its own element in a list
# mainReadings = ["R", "C", "L", "Z"]
# minorReadings = ["X", "D", "Q", "THR" (θ), "ESR"]
mainReadings = ['Z']
minorReadings = ['THR']

try:
    # ser - serial connection
    # freq - user frequencies (Hz)
    # mainReadings - inductance measurements (Ohm, H, C)
    # minorReadings - secondary inducatce measurement (deg, Ohm, None)
    # voltage - the voltage applied between electrodes (V)
    # biasVoltage - the biasvoltage (V)
    voltage = 1 
    biasVoltage = 0
    dataFrame = Experiment(ser, freq, mainReadings, minorReadings, voltage, biasVoltage)
    DeviceDisconnect(ser)
except: 
    print('Test Aborted')
    DeviceDisconnect(ser)     


   Frequency     Z Avg.    Z Std.  Z-THR Avg.  Z-THR Std.
0        110  17.981250  0.006250    -82.7500         0.0
1        332   6.535156  0.000000    -80.1875         0.0
2       1000   2.375977  0.000977    -74.3125         0.0
Test completed ✅


